In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

Загрузим данные для обучения и тестирования

In [3]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

Просмотрим загруженные данные

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Удалим колонки PassengerId, Ticket, Name, которые не планируется использовать для построения модели, а также колонку Cabin, состоящую из пропусков на 80%.

In [8]:
df_train.drop(labels=['PassengerId', 'Cabin', 'Ticket', 'Name'], axis=1, inplace=True)
df_test.drop(labels=['PassengerId' ,'Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

Оценим количество пропусков в данных

In [9]:
df_train.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
df_test.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

Заполним пропуски медианными значениями (Age, Fare) и значениями моды (Embarked)

In [11]:
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_train['Embarked'].fillna(df_train['Embarked'].mode(), inplace=True)
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)

Заменим номинальные признаки Sex и Embarked на числовые

In [12]:
encoder = LabelEncoder()
encoder.fit(df_train["Sex"])
df_train["Sex"] = encoder.transform(df_train["Sex"])
df_test["Sex"] = encoder.transform(df_test["Sex"])
encoder.fit(df_train["Embarked"])
df_train["Embarked"] = encoder.transform(df_train["Embarked"])
df_test["Embarked"] = encoder.transform(df_test["Embarked"])

Нормализуем данные в столбцах Age и Fare

In [13]:
scaler = StandardScaler()
df_train["Age"] = scaler.fit_transform(df_train[["Age"]])
df_train["Fare"] = scaler.fit_transform(df_train[["Fare"]])
df_test["Age"] = scaler.fit_transform(df_test[["Age"]])
df_test["Fare"] = scaler.fit_transform(df_test[["Fare"]])

Построим модель логистической регрессии

In [14]:
y_column = 'Survived'
x_columns = [column for column in df_train.columns if column != 'Survived']
logreg_clf = LogisticRegression()
logreg_clf.fit(df_train[x_columns], df_train[y_column])

LogisticRegression()

Получим предсказание модели для тестовых данных

In [15]:
predict = logreg_clf.predict(df_test[x_columns])

Оценим модель сравнив предсказанные данные с реальными

In [16]:
df_result = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
df_answ = df_result[df_result.PassengerId > 891].Survived
print(f"Accuracy: {accuracy_score(df_answ, predict)}")
print(f"F1 Score: {f1_score(df_answ, predict)}")

Accuracy: 0.9401913875598086
F1 Score: 0.919093851132686


In [17]:
output = pd.DataFrame({'PassengerId': df_result[df_result.PassengerId > 891].PassengerId, 'Survived': predict})

In [18]:
output.to_csv('submission.csv', index=False)